# VISUALISATION DES RESULTATS POUR DTM ET DIM

## **OBJECTIF GENERAL**

**Visualiser les évolutions thématiques**

## **OBJECTIFS SPECIFIQUES**

- Visualiser évolution des groupes de mots au sein des différents thèmes
- Visualiser évolution d'influence des différents thèmes

Import bibliothèques

In [ ]:
import gensim
import tqdm
import json
from gensim import corpora,utils
from gensim.models.wrappers.dtmmodel import DtmModel
from tqdm import tqdm
import time
import numpy as np
from scipy.stats import linregress
import matplotlib.pyplot as plt
import pandas as pd

## 0) Préparation des données

**Fonctions de création du corpus**

In [ ]:
# Récupérer les articles en français, de type article et avec année de publication
def filtre_corpus(Corpus, tokens):
    """Prend en paramètre le Corpus avec textes et métadonnées et retourne le Corpus filtré avec les articles en français et de type article
    + les tokens nettoyés pour chacun d'entre eux et l'année de publication"""
    Corpus_filtre = []
    index_clean = 0
    for index_document in tqdm(range(len(Corpus))) :
    # on ne traite que les articles de type article et en français
        if (Corpus[index_document]['metadata']['typeart'], Corpus[index_document]['metadata']['lang']) == ('article','fr') : 
            Corpus_filtre.append((Corpus[index_document]['metadata']['annee'], tokens[index_clean]))
            index_clean +=1
    return Corpus_filtre

In [ ]:
def tri_annee (Corpus_filtre):
    """On trie le corpus selon les années
        Output : 
        - time_slices = liste où chaque élément contient le nombre d'articles par année
        - articles_par_annee = liste où chaque élément est un tuple du type (année, [liste des tokens des articles de cette année])"""
    time_slices = [] # chaque élément compte le nombre d'articles à chaque time step dans l'ordre croissant des années
    articles_par_annee = {} # contient les tokens des articles pour chaque année de publication
    for element in Corpus_filtre:
        annee = element[0]
        if annee in articles_par_annee : 
            articles_par_annee[annee].append(element[1])
        else : articles_par_annee [annee] = [element[1]]
    
    articles_par_année_sorted = sorted(articles_par_annee.items()) # ordonnement par année
    
    for annee in range(len(articles_par_année_sorted)):
        time_slices.append(len(articles_par_année_sorted[annee][1]))
    return time_slices, articles_par_année_sorted

In [ ]:
class DTMcorpus(corpora.textcorpus.TextCorpus):

    def get_texts(self):
        return self.input

    def __len__(self):
        return len(self.input)

**Préparation des données**

In [ ]:
# CODE POUR ETUDES INTERNATIONALES
Corpus_AE = json.loads(open("Data/corpus_ae.json", "r").read())
tokens_AE= json.loads(open("Data/tokens_ae.json", "r").read())
Corpus_annees_AE = filtre_corpus(Corpus_AE, tokens_AE)
time_slices_AE, articles_par_annee_AE = tri_annee(Corpus_annees_AE)
train_texts_AE = []
for annee in range(len(articles_par_annee_AE)):
    train_texts_AE.append(articles_par_annee_AE[annee][1])

train_texts_AE = [element for sublist in train_texts_AE for element in sublist]
corpus_AE = DTMcorpus(train_texts_AE)

In [ ]:
# CODE POUR ETUDES INTERNATIONALES
Corpus_EI = json.loads(open("Data/corpus_ei.json", "r").read())
tokens_EI = json.loads(open("Data/tokens_ei.json", "r").read())
Corpus_annees_EI = filtre_corpus(Corpus_EI, tokens_EI)
time_slices_EI, articles_par_annee_EI = tri_annee(Corpus_annees_EI)
train_texts_EI = []
for annee in range(len(articles_par_annee_EI)):
    train_texts_EI.append(articles_par_annee_EI[annee][1])
    
train_texts_EI = [element for sublist in train_texts_EI for element in sublist]
corpus_EI = DTMcorpus(train_texts_EI)

In [ ]:
# CODE POUR RELATIONS INDUSTRIELLES
Corpus_RI = json.loads(open("Data/corpus_ri.json", "r").read())
tokens_RI = json.loads(open("Data/tokens_ri.json", "r").read())
Corpus_annees_RI = filtre_corpus(Corpus_RI, tokens_RI)
time_slices_RI, articles_par_annee_RI = tri_annee(Corpus_annees_RI)
train_texts_RI = []
for annee in range(len(articles_par_annee_RI)):
    train_texts_RI.append(articles_par_annee_RI[annee][1])
    
train_texts_RI = [element for sublist in train_texts_RI for element in sublist]
corpus_RI = DTMcorpus(train_texts_RI)

**Récupération des modèles**

In [ ]:
# DTM
dtm_ae_10 = DtmModel.load('Résultats_DTM-DIM/dtm/dtm_ae_10')
dtm_ei_10 = DtmModel.load('Résultats_DTM-DIM/dtm/dtm_ei_10')
dtm_ri_10 = DtmModel.load('Résultats_DTM-DIM/dtm/dtm_ri_10')

# DIM
dim_ae_10 = DtmModel.load('Résultats_DTM-DIM/dim/dim_ae_10')
dim_ei_10 = DtmModel.load('Résultats_DTM-DIM/dim/dim_ei_10')
dim_ri_10 = DtmModel.load('Résultats_DTM-DIM/dim/dim_ri_10')

## 1) PyLDAvis

In [ ]:
import pyLDAvis
pyLDAvis.enable_notebook()
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

**BUG** : temps de run très long (>15min ?)

In [ ]:
doc_topic, topic_term, doc_lengths, term_frequency, vocab = dtm_ae_10.dtm_vis(time=0, corpus=corpus_AE)
vis_wrapper = pyLDAvis.prepare(topic_term_dists=topic_term, doc_topic_dists=doc_topic, doc_lengths=doc_lengths, vocab=vocab, term_frequency=term_frequency)
pyLDAvis.show(vis_wrapper)

## 2)  STATISTIQUES DE TOPIC MODELING

**Probabilité d'une paire mot/thème**

Définition : force d'un mot au sein d'un thème donné. Autrement dit, on mesure la probabilité d'un mot sachant un thème donné. 

In [ ]:
def term_distribution(model, term, topic):
    """Probabilité à chaque année d'une paire mot/thème"""
    word_index = model.id2word.token2id[term]
    topic_slice = np.exp(model.lambda_[topic])
    topic_slice = topic_slice / topic_slice.sum(axis=0)
    return topic_slice[word_index]

In [ ]:
# Exemple
term_distribution(dtm_ae_10, term='économie', topic=0)

**Mesure de variance pour un token**

Définition : indique la variance en probabilité des mots d'un thème au cours du temps. Autrement dit, les mots dont l'importance relative changent le plus au cours du temps pour un thème donné sont les plus porteurs d'information relative au changement de thème.

In [ ]:
def term_variance(model, topic):
    """Donne la variance dans le temps de mots pour un thème donné
       Plus la variance est élevée, plus le mot est d'intérêt"""
    p = np.exp(model.lambda_[topic]) /\
        np.exp(model.lambda_[topic]).sum(axis=0)
    variances = np.var(p, axis=1)
    order = np.argsort(variances)[::-1]
    terms = np.array([term for term, _
                    in sorted(model.id2word.token2id.items(),
                              key=lambda x: x[1])])[order]
    variances = variances[order]
    return list(zip(terms, variances))[:20]

In [ ]:
# Exemple
term_variance(dtm_ae_10, topic=3)

**Pente de variation en probabilité d'un mot au sein d'un thème**

In [ ]:
 def term_slope(model, topic):
        """Mesure d'une pente de variation dans le temps pour les mots composant un thème
        Peut s'interpréter comme une mesure de popularité dans le temps"""
        p = np.exp(model.lambda_[topic]) /\
            np.exp(model.lambda_[topic]).sum(axis=0)
        slopes = np.apply_along_axis(
            lambda y: linregress(x=range(len(y)), y=y).slope, axis=1, arr=p)
        order = np.argsort(slopes)
        terms = np.array([term for term, _
                        in sorted(model.id2word.token2id.items(),
                                    key=lambda x: x[1])])[order]
        slopes = slopes[order]
        return list(zip(terms, slopes))[:20]

In [ ]:
term_slope(dtm_ae_10, topic=0)

## 3) Plots temporels par mot et thème

In [ ]:
def term_distribution(model, term, topic):
    """Probabilité à chaque année d'une paire mot/thème"""
    word_index = model.id2word.token2id[term]
    topic_slice = np.exp(model.lambda_[topic])
    topic_slice = topic_slice / topic_slice.sum(axis=0)
    return topic_slice[word_index]

In [ ]:
time_slices_labels_AE = [element[0] for element in articles_par_annee_AE]
time_slices_labels_EI = [element[0] for element in articles_par_annee_EI]
time_slices_labels_RI = [element[0] for element in articles_par_annee_RI]

In [ ]:
def plot_terms( model, topic, time, time_slice_labels, num_words=5, revue = 'AE', title=None, name=None, hide_y=False, plot=False):
        """Creates a plot of term probabilities over time in a given topic."""
        
        terms = [element[1] for element in model.show_topic(topicid=topic, time=time, num_words=num_words)]
        
        fig, ax = plt.subplots(figsize=(15,5))
        #plt.style.use('fivethirtyeight')
        for term in terms:
            try:
                ax.plot(
                    time_slice_labels, term_distribution(model,term, topic),
                    label=term)
            except KeyError as e : print(repr(e), " --- Le mot ", term, "n'est pas dans le vocabulaire du corpus")
        leg = ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
        plt.xticks(rotation=90)
        if hide_y:
            ax.set_yticklabels([])
        ax.set_ylabel('Probability')
        if title:
            ax.set_title('Evolution des '+ str(num_words) + ' tokens majoritaires en ' + str(time_slice_labels[time]) + ' pour le thème ' +  str(topicid))
        if name:
            fig.savefig('Plots/DTM/'+ revue +
                '/plot_dtm_topic=' + str(topic) + '_time=' + str(time_slice_labels[time]), dpi=300, bbox_extra_artists=(leg,), bbox_inches='tight')
        if not plot:
            plt.close()
        else :
            return fig, ax

**Plots pours AE**

In [ ]:
topics_ids = [x for x in range(10)][5:]
time_slices, time_slices_labels = time_slices_AE, time_slices_labels_AE
times = [x for x in range(len(time_slices_labels))]

# On considère les mots prédominants lors d'une année avec un intervalle de 10 ans entre chaque année
selected_time_slices = [x for x in range(0, len(time_slices), 10)]
for topicid in tqdm(topics_ids):
    for time in selected_time_slices:
        plot_terms(model=dtm_ae_10, topic=topicid, time = time, time_slice_labels=time_slices_labels_AE, topn=5, title='Evolution des 5 tokens majoritaires', name=True, plot=False)

**Plots pours EI**

In [ ]:
topics_ids = [x for x in range(10)]
time_slices, time_slices_labels = time_slices_EI, time_slices_labels_EI
times = [x for x in range(len(time_slices_labels))]

# On considère les mots prédominants lors d'une année avec un intervalle de 10 ans entre chaque année
selected_time_slices = [x for x in range(0, len(time_slices), 10)]
for topicid in tqdm(topics_ids):
    for time in selected_time_slices:
        plot_terms(model=dtm_ei_10, topic=topicid, time = time, time_slice_labels=time_slices_labels, num_words=5, revue='EI', title='Evolution des 5 tokens majoritaires', name=True, plot=False)

**Plots pours RI**

In [ ]:
topics_ids = [x for x in range(10)][6:]
time_slices, time_slices_labels = time_slices_RI, time_slices_labels_RI
times = [x for x in range(len(time_slices_labels))]

# On considère les mots prédominants lors d'une année avec un intervalle de 10 ans entre chaque année
selected_time_slices = [x for x in range(0, len(time_slices), 10)]
for topicid in tqdm(topics_ids):
    for time in selected_time_slices:
        plot_terms(model=dtm_ri_10, topic=topicid, time = time, time_slice_labels=time_slices_labels, num_words=5, revue='RI', title='Evolution des 5 tokens majoritaires', name=True, plot=False)

In [ ]:
# Récupérer les mots les plus significatifs à un instant t
time = 0
topicid = 9
time_slices_labels_AE[time]

## 4) Moyenne des tokens sur toute la période de publication

In [ ]:
def tokens_by_topics (model = dtm_ae_10, time_slices= time_slices_AE):
    tokens_by_topics = dict.fromkeys((x for x in range(10)),{})
    for topic in tqdm(range(10)):
        tokens={}
        for time in range(len(time_slices)):
            for element in model.show_topic(topicid=topic,time=time):
                if element[1] in tokens:
                    tokens[element[1]] += element[0]
                else : 
                    tokens[element[1]] = element[0]
        tokens = {k: v /len(time_slices)  for k, v in tokens.items()}
        tokens_by_topics[topic] = tokens
    return tokens_by_topics

In [ ]:
# Print pour stocker dans excel
def print_tokens_by_topics(tokens_by_topics):
    for topic in range(10):
        print('Topic ' + str(topic))
        d = sorted(tokens_by_topics[topic].items(), key=lambda x: x[1], reverse=True)
        i=0
        for element in d:
            while i < 10:
                print(d[i][0])
                i+=1
        print('\n')

**AE**

In [ ]:
tokens_by_topics_AE = tokens_by_topics(dim_ae_10, time_slices_AE)

In [ ]:
print_tokens_by_topics(tokens_by_topics=tokens_by_topics_AE)

**EI**

In [ ]:
tokens_by_topics_EI = tokens_by_topics(dim_ei_10, time_slices_EI)

In [ ]:
print_tokens_by_topics(tokens_by_topics=tokens_by_topics_EI)

**RI**

In [ ]:
tokens_by_topics_RI = tokens_by_topics(dim_ri_10, time_slices_RI)

In [ ]:
print_tokens_by_topics(tokens_by_topics=tokens_by_topics_RI)

**Tableau avec les n termes plus représentatif d'un thème en fonction du temps**

In [ ]:
time_slices_labels_AE = [element[0] for element in articles_par_annee_AE]
time_slices_labels_EI = [element[0] for element in articles_par_annee_EI]
time_slices_labels_RI = [element[0] for element in articles_par_annee_RI]

In [ ]:
def top_term_table(model, topic, slices, time_slice_label, topn=30, print_=True):
    """Returns a dataframe with the top n terms in the topic for each of
    the given time slices."""
    data = {}
    for time_slice in slices:
        time = time_slice_label.index(str(time_slice))
        data[time_slice] = [
            term for p, term
            in model.show_topic(topic, time=time, topn=topn)
        ]
    if print_:
        print(pd.DataFrame(data))
        
    return pd.DataFrame(data)

In [ ]:
slices = [1960,1970,1980, 1990, 2000,2010]
for topic in range(10):
    # DTM
    top_table_ae_topic = top_term_table(dtm_ae_10, topic=topic,slices=slices,time_slice_label= time_slices_labels_AE)
    top_table_ae_topic.to_pickle('Plots/DTM/AE/Tables Top Mots par an/top_terms_topic='+str(topic))
    # DIM 
    top_table_ae_topic = top_term_table(dim_ae_10, topic=topic,slices=slices,time_slice_label= time_slices_labels_AE)
    top_table_ae_topic.to_pickle('Plots/DIM/AE/Tables Top Mots par an/top_terms_topic='+str(topic))

In [ ]:
slices = [1970,1980, 1990, 2000,2010]
for topic in range(10):
    # DTM
    top_table_ei_topic = top_term_table(dtm_ei_10, topic=topic,slices=slices,time_slice_label= time_slices_labels_EI)
    top_table_ei_topic.to_pickle('Plots/DTM/EI/Tables Top Mots par an/top_terms_topic='+str(topic))
    # DIM
    top_table_ei_topic = top_term_table(dim_ei_10, topic=topic,slices=slices,time_slice_label= time_slices_labels_EI)
    top_table_ei_topic.to_pickle('Plots/DIM/EI/Tables Top Mots par an/top_terms_topic='+str(topic))

In [ ]:
slices = [1950,1960, 1970,1980, 1990, 2000,2010]
for topic in range(10):
    # DTM
    top_table_ri_topic = top_term_table(dtm_ri_10, topic=topic,slices=slices,time_slice_label= time_slices_labels_RI)
    top_table_ri_topic.to_pickle('Plots/DTM/RI/Tables Top Mots par an/top_terms_topic='+str(topic))
    # DIM
    top_table_ri_topic = top_term_table(dim_ri_10, topic=topic,slices=slices,time_slice_label= time_slices_labels_RI)
    top_table_ri_topic.to_pickle('Plots/DIM/RI/Tables Top Mots par an/top_terms_topic='+str(topic))

In [ ]:
top_table_ri_topic = top_term_table(dim_ri_10, topic=topic,slices=slices,time_slice_label= time_slices_labels_RI)

Récupération des tables

In [ ]:
for i in range(10):
    df = pd.read_pickle('Plots/DIM/AE/Tables Top Mots par an/top_terms_topic=' +str(i))
    print("Thème ", str(i) + "\n",  df,"\n", "-"*40 )

**Résumé pour tous les thèmes et par année**

In [ ]:
def summary(model, slices, time_slices_labels, topn=10):
    """Prints a summary of all the topics"""
    for topic in range(model.num_topics):
        print('Topic %d' % topic)
        print(top_term_table(model, topic=topic,slices=slices,time_slice_label= time_slices_labels))
        print()

In [ ]:
slices = [1960,1970,1980, 1990, 2000,2010]
summary(model=dtm_ae_10,slices=slices, time_slices_labels=time_slices_labels_AE)

### Evolution temporelle des mots-clés

Idée : étudier la répartition de certains mots-clés en fonction du temps (cf baseline pour modèle dynamique)

In [ ]:
#### Récupérer les métadonnées et tokens pour les articles
def filtre_corpus_année(Corpus, tokens):
    """Prend en paramètre le Corpus avec textes et métadonnées et retourne le Corpus filtré avec les articles en français et de type article
    + les tokens nettoyés pour chacun d'entre eux"""
    Corpus_filtre = []
    index_clean = 0
    for index_document in tqdm_notebook(range(len(Corpus))) :
    # on ne traite que les articles de type article et en français
        if (Corpus[index_document]['metadata']['typeart'], Corpus[index_document]['metadata']['lang']) == ('article','fr') : 
            Corpus_filtre.append((Corpus[index_document]['URL'], Corpus[index_document]['metadata']['title'], 
                                  Corpus[index_document]['metadata']['annee'], tokens[index_clean]))
            index_clean +=1
    return Corpus_filtre

In [ ]:
%store -r Corpus_EI
%store -r tokens_bigrams_Corpus_LDA_EI_clean

In [ ]:
Corpus_EI_LDA_metadata_annee = filtre_corpus_année(Corpus_EI, tokens_bigrams_Corpus_LDA_EI_clean)

In [ ]:
# compte_frequence_EI contient pour chaque article un quadruplet de la forme : (URL, titre, annee, (token , occurence du token))
compte_frequence_EI_annee = []
for article in Corpus_EI_LDA_metadata_annee:
    compte_frequence_EI_annee.append((article[0], article[1], article[2], compte_frequences(article[3])))

In [ ]:
def find_string_annee(string, string2 ='', double_check=False):
    """Chercher un token d'intérêt et retourner le quadruplet de compte_frequence_AE le cas échéant"""
    articles_par_années ={}
    articles_with_string = []
    for article in compte_frequence_EI_annee:
        for element in article[3]:
            if string == element[0]:
                if double_check == True: #option pour vérifier la présence simultanée de 2 string dans le même texte
                    for element in article[3] :
                        if string2 == element[0]:
                            try: articles_par_années[article[2]].appendl(article)
                            except: articles_par_années[article[2]] = [article]
                            break
                else : 
                    try: articles_par_années[article[2]].append(article)
                    except: articles_par_années[article[2]] = [article]
                    break
    return articles_par_années

In [ ]:
import matplotlib.pyplot as plt
def plot_repartition_mot(test_string):
    annees = sorted([annee for annee in test_string])
    comptes = [len(test_string[annee]) for annee in annees]
    fig = plt.figure(figsize=(10,8))
    plt.xticks(rotation=90)
    plt.bar(annees,comptes)
    plt.show()

In [ ]:
test_string = find_string_annee('identité', '', double_check=False)
plot_repartition_mot(test_string)

slices = [1960,1970,1980, 1990, 2000,2010]
summary(model=model,slices=slices, time_slices_labels=time_slices_labels_AE)

# DIM

L'influence est une fonction avec 3 variables :
- article
- thème
- année

In [ ]:
time_slices_labels_AE = [element[0] for element in articles_par_annee_AE]
time_slices_labels_EI = [element[0] for element in articles_par_annee_EI]
time_slices_labels_RI = [element[0] for element in articles_par_annee_RI]

In [ ]:
def top_term_table(model, topic, slices, time_slice_label, topn=10):
    """Returns a dataframe with the top n terms in the topic for each of
    the given time slices."""
    data = {}
    for time_slice in slices:
        time = time_slice_label.index(str(time_slice))
        data[time_slice] = [
            term for p, term
            in model.show_topic(topic, time=time, topn=topn)
        ]
    return pd.DataFrame(data)

In [ ]:
# Récupérer les articles en français, de type article et avec année de publication
def filtre_corpus_metadata(Corpus, tokens, time_slices_labels = time_slices_labels_AE):
    """Prend en paramètre le Corpus avec textes et métadonnées et retourne le Corpus filtré avec les articles en français et de type article
    + les tokens nettoyés pour chacun d'entre eux et l'année de publication"""
    Corpus_filtre = dict.fromkeys(time_slices_labels)
    index_clean = 0
    for index_document in tqdm(range(len(Corpus))) :
    # on ne traite que les articles de type article et en français
        if (Corpus[index_document]['metadata']['typeart'], Corpus[index_document]['metadata']['lang']) == ('article','fr') : 
            try :
                Corpus_filtre[Corpus[index_document]['metadata']['annee']].append((Corpus[index_document]['URL'], Corpus[index_document]['metadata']['title'],tokens[index_clean])) 
            except:
                Corpus_filtre[Corpus[index_document]['metadata']['annee']] =[(Corpus[index_document]['URL'], Corpus[index_document]['metadata']['title'],tokens[index_clean])]
            index_clean +=1
    return Corpus_filtre

In [ ]:
def get_documents_influents_par_année (topic=0, time_slices=time_slices_AE, articles_par_annee = articles_par_annee_AE, time_slices_labels = time_slices_labels_AE):
    """Pour un thème donné, retourne un dictionnaire du type (année, (influence, tokens du documents)) trié par ordre décroissant d'influence à chaque année"""
    
    # Récupération des valeurs d'influence pour les articles à chaque temps t
    influents_par_année = []
    for time_slice in range(len(time_slices)): # chaque année
        document_influent =[]
        for document in range(time_slices[time_slice]): # chaque document publié cette année-là
            document_influent.append(model.influences_time[time_slice][document][topic])
        influents_par_année.append(document_influent)
        
    # Lien entre influence et liste de tokens de l'article
    dico_influences = {}
    for time_slice in range(len(time_slices_AE)):
        dico = []
        for index in range(len(articles_par_annee[time_slice][1])):
            dico.append((influents_par_année[time_slice][index],articles_par_annee[time_slice][1][index]))
        dico_influences[time_slices_labels[time_slice]] = dico
        
    # tri par ordre décroissant d'influence à chaque année
    for key, value in dico_influences.items(): 
        value.sort(key=lambda tup: tup[0], reverse=True)
        
    return dico_influences

In [ ]:
def get_influence_metadata (dico_influences, corpus_metadata, time_slices_labels = time_slices_labels_AE):
    """Associer influence avec métadonnées de l'article
    Output : dico du type (année : [[influence, (URL, titre, tokens)], ...])"""
    dico_influence_metadata = dict.fromkeys(time_slices_labels)
    for année, liste in corpus_metadata.items():
        for element in liste :
            for element_2 in dico_influences[année]:
                if element[2] == element_2[1]:
                    try : 
                        dico_influence_metadata[année].append((element_2[0], element))
                    except :
                        dico_influence_metadata[année] = [(element_2[0], element)]
    
    # tri par ordre décroissant d'influence à chaque année
    for key, value in dico_influence_metadata.items(): 
        value.sort(key=lambda tup: tup[0], reverse=True)
        
    return dico_influence_metadata

**AE**

In [ ]:
model = DtmModel.load('Résultats_DTM-DIM/dim/dim_ae_10')

In [ ]:
# Préparation du corpus
corpus_ae_metadata = filtre_corpus_metadata(Corpus_AE,tokens_AE, time_slices_labels_AE)

**VISUALISATION DES DOCUMENTS INFLUENTS par rapport aux documents où le THEME est prédominant**

In [ ]:
def summary_et_influents(model, dico_influence_metadata, slices, time_slices_labels, topn=10, topic=0):
    """Affiche les mots clés prédominants pour un thème et les documents influents à chaque temps t"""
    print('Topic %d' % topic)
    print(top_term_table(model, topic=topic,slices=slices,time_slice_label= time_slices_labels))
    print()
    for année in slices:
        print('Documents les plus influents en', str(année))
        for element in dico_influence_metadata[str(année)][:5]:
            print(element[1][1])
    
        print()

In [ ]:
for element in dico_influence_metadata_ae['1980']:
    print(element[1][1])

In [ ]:
topic=2
dico_influences_AE = get_documents_influents_par_année(topic=topic, time_slices=time_slices_AE, articles_par_annee = articles_par_annee_AE, time_slices_labels = time_slices_labels_AE)
dico_influence_metadata_ae = get_influence_metadata(time_slices_labels = time_slices_labels_AE, dico_influences = dico_influences_AE, corpus_metadata = corpus_ae_metadata)
summary_et_influents(model=model, slices = [1960,1970,1980, 1990, 2000,2010], time_slices_labels=time_slices_labels_AE, topic=topic, dico_influence_metadata = dico_influence_metadata_ae)

In [ ]:
topic=1
dico_influences_AE = get_documents_influents_par_année(topic=topic, time_slices=time_slices_AE, articles_par_annee = articles_par_annee_AE, time_slices_labels = time_slices_labels_AE)
dico_influence_metadata_ae = get_influence_metadata(time_slices_labels = time_slices_labels_AE, dico_influences = dico_influences_AE, corpus_metadata = corpus_ae_metadata)
summary_et_influents(model, slices = [1960,1970,1980, 1990, 2000,2010], time_slices_labels=time_slices_labels_AE, topic=topic)

In [ ]:
topic=3
dico_influences_AE = get_documents_influents_par_année(topic=topic, time_slices=time_slices_AE, articles_par_annee = articles_par_annee_AE, time_slices_labels = time_slices_labels_AE)
dico_influence_metadata_ae = get_influence_metadata(time_slices_labels = time_slices_labels_AE, dico_influences = dico_influences_AE, corpus_metadata = corpus_ae_metadata)
summary_et_influents(model, slices = [1960,1970,1980, 1990, 2000,2010], time_slices_labels=time_slices_labels_AE, topic=topic)

**Plot temporel de l'influence globale d'un thème**

In [ ]:
# Checker le document le plus influent au temps t=time pour chaque topic
def create_dico_topic(model, time_slices):
    dico_topic = {}
    for topic in range(model.num_topics):
        dico_topic[topic] = []
        for time in range(len(time_slices)):
            influence_time = 0
            for document in range(time_slices[time]):
                # on somme les influences des différents documents de cette slice
                influence_time +=  model.influences_time[time][document][topic]
            # normalisation par le nombre de documents à ce temps t
            dico_topic[topic].append(influence_time/time_slices[time])
    return dico_topic

In [ ]:
def plot_topics_trend(topics, dico_topic, time_slice_labels, revue = 'AE', title=None, name=None, plot=True):
        """Trace l'évolution de l'influence des thèmes"""        
        fig, ax = plt.subplots(figsize=(15,5))
        for topic in topics:
            ax.plot(time_slice_labels, dico_topic[topic], label=topic)
        
        leg = ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
        plt.xticks(rotation=90)
        
        ax.set_ylabel('Influence')
        if title:
            ax.set_title('Evolution de l\'influence des thèmes pour le modèle LDA à 10 thèmes sur la revue '+ revue)
        if name:
            fig.savefig('Plots/DIM/'+ revue +
                '/Plots temporels/evolution_influence_thèmes_'+ revue +'_topics=' + str(topics) , dpi=300, bbox_extra_artists=(leg,), bbox_inches='tight')
        if not plot:
            plt.close()
        else :
            return fig, ax

**AE**

In [ ]:
dico_topic_AE = create_dico_topic(model=dim_ae_10, time_slices=time_slices_AE)

In [ ]:
plot_topics_trend(topics=[x for x in range(10)], dico_topic=dico_topic_AE, time_slice_labels=time_slices_labels_AE, revue = 'AE', title=True, name=True, plot=True)

In [ ]:
plot_topics_trend(topics=[2,4], time_slice_labels=time_slices_labels_AE, revue = 'AE', title=True, name=True, plot=True)

In [ ]:
plot_topics_trend(topics=[0], time_slice_labels=time_slices_labels_AE, revue = 'AE', title=True, name=True, plot=True)

In [ ]:
dim_ae_10.show_topic(topicid=2,time=30,topn=10)

In [ ]:
plot_topics_trend(topics=[3,5,6,7,8,9],dico_topic=dico_topic_AE, time_slice_labels=time_slices_labels_AE, revue = 'AE', title=True, name=True, plot=True)

In [ ]:
dim_ae_10.show_topic(topicid=3,time=30,topn=10)

In [ ]:
dim_ae_10.show_topic(topicid=5,time=30,topn=10)

In [ ]:
dim_ae_10.show_topic(topicid=6,time=60,topn=10)

**EI**

In [ ]:
dico_topic_EI = create_dico_topic(model=dim_ei_10, time_slices=time_slices_EI)

In [ ]:
plot_topics_trend(topics=[x for x in range(10)], dico_topic=dico_topic_EI, time_slice_labels=time_slices_labels_EI, revue = 'EI', title=True, name=True, plot=True)

In [ ]:
plot_topics_trend(topics=[4,9], dico_topic=dico_topic_EI, time_slice_labels=time_slices_labels_EI, revue = 'EI', title=True, name=None, plot=True)

In [ ]:
dim_ei_10.show_topic(topicid=4,time=30,topn=10)

In [ ]:
dim_ei_10.show_topic(topicid=9,time=40,topn=10)

**RI**

In [ ]:
dico_topic_RI = create_dico_topic(model=dim_ri_10, time_slices=time_slices_RI)

In [ ]:
plot_topics_trend(topics=[x for x in range(10)], dico_topic=dico_topic_RI, time_slice_labels=time_slices_labels_RI, revue = 'RI', title=True, name=True, plot=True)

In [ ]:
# Vérifier le document le plus influent au temps t=time pour chaque topic
time = 30
influences = [-1 for _ in range(10)]
documents = [0 for _ in range(10)]
for topic in range(10):
    for document in range(time_slices_AE[time]):
        if dim_ae_10.influences_time[time][document][topic] > influences[topic]:
            influences[topic] = dim_ae_10.influences_time[time][document][topic]
            documents[topic] = document

In [ ]:
documents, influences